In [13]:
##### do NOT remove STOPWORD before LIWC #####
##### do NOT do lemma / stem before LIWC #####

import liwc
import re
from collections import Counter
import nltk
import csv
import statistics
import time
import numpy as np
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# from pycorenlp import StanfordCoreNLP
logodds_path = "../01-logodds-topbot1500-R1-nostops.txt"
tfidf_path   = "../02-tfidf-depcon3000-R1-nostops.txt"
lexicon_path = "../03-depression-lexicon.txt"
sumatt_path  = "../04-top-sum-attention-3000.txt"
topatt_path  = "../topatt_masked_words_train.txt"
nn_path      = "../nn_masked_words_train.txt"


with open(logodds_path) as f:
    logodds = f.readlines()
    logodds = [ word[:-1] for word in logodds]
    print(len(logodds))
    # print(logodds)

    # # tokenize the lexicon and keep the input_ids of each word together in a list 
    # keyword_input_ids = [ torch.tensor((tokenizer(lex).input_ids)[1:-1]) for lex in lexicon ]
    # print(len(keyword_input_ids))
    # print(keyword_input_ids)
    # keyword_list = keyword_input_ids
    # assert keyword_list is not None
    
with open(tfidf_path) as f:
    tfidf = f.readlines()
    tfidf = [ word[:-1] for word in tfidf]
    print(len(tfidf))
    # print(tfidf)
    
with open(lexicon_path) as f:
    lexicon = f.readlines()
    lexicon = [ word[:-1] for word in lexicon]
    print(len(lexicon))
    # print(lexicon)
    
with open(sumatt_path) as f:
    sumatt = f.readlines()
    sumatt = [ word[:-1] for word in sumatt]
    print(len(sumatt))
    # print(sumatt)
    
with open(topatt_path) as f:
    topatt = f.readlines()
    topatt = [ word[:-1] for word in topatt]
    print(len(topatt))
    
with open(nn_path) as f:
    nn = f.readlines()
    nn = [ word[:-1] for word in nn]
    print(len(nn))

3000
3000
112
3000
26526
22966


[nltk_data] Downloading package stopwords to /lustre-
[nltk_data]     home/gpu/home/users/jakrapop.a/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to /lustre-
[nltk_data]     home/gpu/home/users/jakrapop.a/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
sumatt_tokens = [word for word in topatt if '#' in word]
sumatt_words = [word for word in topatt if '#' not in word]
print(len(sumatt_tokens))
print(len(sumatt_words))

4772
21754


# LIWC

In [15]:
parse, category_names = liwc.load_token_parser('./LIWC2007_English_copy.dic')

#### Append LIWC count of each User in to each category list 
liwc_dict = {}
for cat in category_names:
    liwc_dict[cat] = []
print(liwc_dict)

{'funct': [], 'pronoun': [], 'ppron': [], 'i': [], 'we': [], 'you': [], 'shehe': [], 'they': [], 'ipron': [], 'article': [], 'verb': [], 'auxverb': [], 'past': [], 'present': [], 'future': [], 'adverb': [], 'preps': [], 'conj': [], 'negate': [], 'quant': [], 'number': [], 'swear': [], 'social': [], 'family': [], 'friend': [], 'humans': [], 'affect': [], 'posemo': [], 'negemo': [], 'anx': [], 'anger': [], 'sad': [], 'cogmech': [], 'insight': [], 'cause': [], 'discrep': [], 'tentat': [], 'certain': [], 'inhib': [], 'incl': [], 'excl': [], 'percept': [], 'see': [], 'hear': [], 'feel': [], 'bio': [], 'body': [], 'health': [], 'sexual': [], 'ingest': [], 'relativ': [], 'motion': [], 'space': [], 'time': [], 'work': [], 'achieve': [], 'leisure': [], 'home': [], 'money': [], 'relig': [], 'death': [], 'assent': [], 'nonfl': [], 'filler': []}


In [16]:
liwc_counts = Counter(category for token in sumatt_words for category in parse(token))

for key, value in liwc_counts.items():
    print(key, value)
    liwc_dict[key] = value
        
print(liwc_dict)

for key, value in liwc_dict.items():
    print(value)

funct 410
conj 26
cogmech 1162
incl 27
pronoun 65
ppron 35
i 7
excl 22
article 4
you 8
social 623
ipron 30
adverb 69
leisure 399
verb 259
auxverb 35
present 116
preps 63
affect 1454
posemo 702
assent 21
negemo 740
anger 295
bio 627
sexual 83
swear 36
cause 220
they 5
number 80
time 389
relativ 1130
humans 68
nonfl 9
shehe 9
past 127
tentat 164
work 695
achieve 454
space 415
relig 239
health 224
percept 465
see 154
quant 108
negate 13
hear 103
body 217
we 6
motion 310
future 10
death 86
family 63
friend 53
ingest 142
feel 133
certain 110
money 276
insight 395
home 139
discrep 64
inhib 228
sad 149
anx 152
filler 2
{'funct': 410, 'pronoun': 65, 'ppron': 35, 'i': 7, 'we': 6, 'you': 8, 'shehe': 9, 'they': 5, 'ipron': 30, 'article': 4, 'verb': 259, 'auxverb': 35, 'past': 127, 'present': 116, 'future': 10, 'adverb': 69, 'preps': 63, 'conj': 26, 'negate': 13, 'quant': 108, 'number': 80, 'swear': 36, 'social': 623, 'family': 63, 'friend': 53, 'humans': 68, 'affect': 1454, 'posemo': 702, 'negemo

In [ ]:
# # Count total number of tokens that liwc has seen
# total_tokens = 0

# for text in text_all:
#     total_tokens += len(word_tokenize(text))
    
# print('total_tokens : ', total_tokens)